In [1]:
# Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import fiona
import io
import json
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import datetime
import matplotlib.dates as mdates
import matplotlib as mpl

In [2]:
# Loading the data
car_raw_df = pd.read_csv('./files/car-assignments.csv')
card_raw_df = pd.read_csv('./files/cc_data.csv')
loyal_raw_df = pd.read_csv('./files/loyalty_data.csv')
gps_raw_df = pd.read_csv('./files/gps.csv')

stops_3min_001thresh_df = pd.read_json('stops_3min_0.001thresh.txt', lines=True)
time_3min_001thresh_df = pd.read_csv('stops_timestamps_3min_0.001thresh.csv')

city_map = plt.imread('./files/city_map.png')

print(car_raw_df.head(0))
print(card_raw_df.head(0))
print(loyal_raw_df.head(0))
print(gps_raw_df.head(0))
print(stops_3min_001thresh_df.head(0))
print(time_3min_001thresh_df.head(0))

Empty DataFrame
Columns: [LastName, FirstName, CarID, CurrentEmploymentType, CurrentEmploymentTitle]
Index: []
Empty DataFrame
Columns: [timestamp, location, price, FirstName, LastName]
Index: []
Empty DataFrame
Columns: [timestamp, location, price, FirstName, LastName]
Index: []
Empty DataFrame
Columns: [Timestamp, id, lat, long]
Index: []
Empty DataFrame
Columns: [id, long, lat, count]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, stop_id, car_id, timestamp]
Index: []


In [3]:
# Data cleaning and modifying

time_3min_001thresh_df['stop_id'] = time_3min_001thresh_df['stop_id'].astype(int)
time_3min_001thresh_df['car_id'] = time_3min_001thresh_df['car_id'].astype(int)
stops_3min_001thresh_df = stops_3min_001thresh_df.rename(columns={'id': 'stop_id'})

car_stop_df = time_3min_001thresh_df.merge(stops_3min_001thresh_df, on='stop_id', how='inner')
car_stop_df['timestamp'] = pd.to_datetime(car_stop_df['timestamp'])
car_stop_df['hour'] = car_stop_df['timestamp'].dt.strftime('%H').astype(int)
car_stop_df['day'] = car_stop_df['timestamp'].dt.strftime('%d').astype(int)

gps_raw_df['Timestamp'] = pd.to_datetime(gps_raw_df['Timestamp'])
gps_raw_df['hour'] = gps_raw_df['Timestamp'].dt.strftime('%H').astype(int)
gps_raw_df['day'] = gps_raw_df['Timestamp'].dt.strftime('%d').astype(int)

card_raw_df['timestamp'] = card_raw_df['timestamp'] = pd.to_datetime(card_raw_df['timestamp'])
card_raw_df['hour'] = card_raw_df['timestamp'].dt.strftime('%H').astype(int)
card_raw_df['day'] = card_raw_df['timestamp'].dt.strftime('%d').astype(int)

loyal_raw_df['timestamp'] = loyal_raw_df['timestamp'] = pd.to_datetime(loyal_raw_df['timestamp'])
loyal_raw_df['day'] = loyal_raw_df['timestamp'].dt.strftime('%d').astype(int)

car_stop_df.loc[car_stop_df['car_id'] == 101, 'car_id'] = 36
car_stop_df.loc[car_stop_df['car_id'] == 104, 'car_id'] = 37
car_stop_df.loc[car_stop_df['car_id'] == 105, 'car_id'] = 38
car_stop_df.loc[car_stop_df['car_id'] == 106, 'car_id'] = 39
car_stop_df.loc[car_stop_df['car_id'] == 107, 'car_id'] = 40

gps_raw_df.loc[gps_raw_df['id'] == 101, 'id'] = 36
gps_raw_df.loc[gps_raw_df['id'] == 104, 'id'] = 37
gps_raw_df.loc[gps_raw_df['id'] == 105, 'id'] = 38
gps_raw_df.loc[gps_raw_df['id'] == 106, 'id'] = 39
gps_raw_df.loc[gps_raw_df['id'] == 107, 'id'] = 40

car_raw_df['name'] = car_raw_df['FirstName'] + ' ' + car_raw_df['LastName']
card_raw_df['name'] = card_raw_df['FirstName'] + ' ' + card_raw_df['LastName']
loyal_raw_df['name'] = loyal_raw_df['FirstName'] + ' ' + loyal_raw_df['LastName']

gps_raw_df = gps_raw_df.sort_values(by=['id', 'Timestamp'])
gps_raw_df['timedelta'] = gps_raw_df['Timestamp'].diff()

In [4]:
# Name / id dict
name_id_dict = {}
id_name_dict = {}

for i, name in zip(car_raw_df['CarID'], car_raw_df['name']):
    try:
        i = int(i)
        name_id_dict[name] = i
        id_name_dict[i] = name
    except:
        name_id_dict[name] = 'Truckdriver'
        id_name_dict[36] = 'Truckdriver'
        id_name_dict[37] = 'Truckdriver'
        id_name_dict[38] = 'Truckdriver'
        id_name_dict[39] = 'Truckdriver'
        id_name_dict[40] = 'Truckdriver'

card_raw_df['car_id'] = card_raw_df['name'].map(name_id_dict)
loyal_raw_df['car_id'] = loyal_raw_df['name'].map(name_id_dict)

In [5]:
# Interactive function
def interactive_map(cars, time_of_day, dates, show_stops, show_paths, show_card, show_loyal, delta, global_stops):
    car_ids = set()
    _range = False
    i = 0
    r = 0
    for txt in cars:
        try:
            j = int(txt)
            if (i > 0):
                i = i*10 + j
            else:
                i = j
        except:
            if (txt == ' '):
                if (_range):
                    car_ids.update(np.arange(r,i).tolist())
                    _range = False
                car_ids.add(i)
                i = 0
            elif (txt == '-'):
                r = i
                i = 0
                _range = True
    
    xMin = 24.82403
    xMax = 24.909965
    yMin = 36.045015
    yMax = 36.094918
    
    fig, ax = plt.subplots(figsize=[25, 25])
    
    if (global_stops):
        geo_stops = gpd.GeoDataFrame(stops_3min_001thresh_df, geometry=gpd.points_from_xy(stops_3min_001thresh_df['long'], stops_3min_001thresh_df['lat']))
        geo_stops.plot(ax=ax, markersize=geo_stops['count']*10, color='purple', alpha=0.5)
    
    ax.set_aspect('equal')
    ax.set_xlim([xMin, xMax])
    ax.set_ylim([yMin, yMax])
    ax.imshow(city_map, extent=[xMin, xMax, yMin, yMax])
    
    # Filter data
    if (show_stops):
        chosen_stops_data = gps_raw_df.loc[gps_raw_df['id'].isin(car_ids)]
        chosen_stops_data = chosen_stops_data[(chosen_stops_data['hour'] >= time_of_day[0]) &  (chosen_stops_data['hour'] <= time_of_day[1])]
        chosen_stops_data = chosen_stops_data[(chosen_stops_data['day'] >= dates[0]) &  (chosen_stops_data['day'] <= dates[1])]
        chosen_stops_data = chosen_stops_data[chosen_stops_data['timedelta'] >= pd.Timedelta('0 days 00:{}:00'.format(delta))]
        
    if (show_paths):
        chosen_paths_data = gps_raw_df.loc[gps_raw_df['id'].isin(car_ids)]
        chosen_paths_data = chosen_paths_data[(chosen_paths_data['hour'] >= time_of_day[0]) &  (chosen_paths_data['hour'] <= time_of_day[1])]
        chosen_paths_data = chosen_paths_data[(chosen_paths_data['day'] >= dates[0]) &  (chosen_paths_data['day'] <= dates[1])]
    
    if (show_card):
        chosen_card_data = card_raw_df.loc[card_raw_df['car_id'].isin(car_ids)]
        chosen_card_data = chosen_card_data[(chosen_card_data['hour'] >= time_of_day[0]) &  (chosen_card_data['hour'] <= time_of_day[1])]
        chosen_card_data = chosen_card_data[(chosen_card_data['day'] >= dates[0]) &  (chosen_card_data['day'] <= dates[1])]
        
    if (show_loyal):
        chosen_loyal_data = loyal_raw_df.loc[loyal_raw_df['car_id'].isin(car_ids)]
        chosen_loyal_data = chosen_loyal_data[(chosen_loyal_data['day'] >= dates[0]) &  (chosen_loyal_data['day'] <= dates[1])]
    
    active_cars = set()
    # Plot different colors for different ids
    cmap = plt.cm.rainbow(np.linspace(0,1,len(car_ids)))
    for _id, c in zip(car_ids, cmap):
        if (show_stops):
            one_stops_car = chosen_stops_data.loc[chosen_stops_data['id'] == _id]
            if(not one_stops_car.empty):
                active_cars.add(_id)
            one_stops_car = one_stops_car.sort_values(by=['Timestamp'])
            plt.plot(one_stops_car['long'], one_stops_car['lat'], c=c)
            
        if (show_paths):
            geo_stops = gpd.GeoDataFrame(stops_3min_001thresh_df, geometry=gpd.points_from_xy(stops_3min_001thresh_df['long'], stops_3min_001thresh_df['lat']))
            one_paths_car = chosen_paths_data.loc[chosen_paths_data['id'] == _id]
            if(not one_paths_car.empty):
                active_cars.add(_id)
            one_paths_car = one_paths_car.sort_values(by=['Timestamp'])
            plt.plot(one_paths_car['long'], one_paths_car['lat'], c=c)
        if (show_card & show_loyal):
            plt.text(xMin-0.005, yMax, chosen_card_data.to_string(index=False, columns=['car_id', 'name', 'location'], col_space=20), size=15, c=c)
            plt.text(xMin+0.023, yMax, chosen_card_data.to_string(index=False, columns=['timestamp', 'price'], col_space=15), size=15, c=c)
            plt.text(xMin+0.04, yMax, chosen_loyal_data.to_string(index=False, columns=['location'], col_space=20), size=15, c=c)
            plt.text(xMin+0.05, yMax, chosen_loyal_data.to_string(index=False, columns=['timestamp', 'price'], col_space=20), size=15, c=c)
        elif (show_card):
            plt.text(xMin-0.005, yMax, chosen_card_data.to_string(index=False, columns=['car_id', 'name', 'location'], col_space=20), size=15, c=c)
            plt.text(xMin+0.023, yMax, chosen_card_data.to_string(index=False, columns=['timestamp', 'price'], col_space=15), size=15, c=c)
        elif (show_loyal):
            plt.text(xMin-0.005, yMax, chosen_loyal_data.to_string(index=False, columns=['car_id', 'name', 'location'], col_space=20), size=15, c=c)
            plt.text(xMin+0.023, yMax, chosen_loyal_data.to_string(index=False, columns=['timestamp', 'price'], col_space=15), size=15, c=c)

        
    plt.show()
    
    print('Active cars: ', active_cars)

In [6]:
# Interactive visualization
car_text = widgets.Text(
    value='1 ',
    description='Car ids:',
    disabled=False
)

time_slider = widgets.IntRangeSlider(
    value=[0, 23],
    min=0,
    max=23,
    step=1,
    description='Time of day:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

date_slider = widgets.IntRangeSlider(
    value=[6, 19],
    min=6,
    max=19,
    step=1,
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

show_stops = widgets.Checkbox(
    value=False,
    description='Stops',
    disabled=False,
    indent=False
)

show_paths = widgets.Checkbox(
    value=True,
    description='Paths',
    disabled=False,
    indent=False
)

show_card = widgets.Checkbox(
    value=False,
    description='Card info',
    disabled=False,
    indent=False
)

show_loyal = widgets.Checkbox(
    value=False,
    description='Loyalty info',
    disabled=False,
    indent=False
)

global_stops = widgets.Checkbox(
    value=False,
    description='Show stops for all data',
    disabled=False,
    indent=False
)

timedelta_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=59,
    step=1,
    description='Stop thresh:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)



ui1 = widgets.HBox([show_stops, show_paths, show_card, show_loyal])
ui2 = widgets.HBox([car_text, time_slider, date_slider])
ui3 = widgets.HBox([timedelta_slider, global_stops])
vis = widgets.interactive_output(interactive_map, {'cars': car_text, 'time_of_day': time_slider, 'dates': date_slider,
                                                  'show_stops': show_stops, 'show_paths': show_paths,
                                                   'show_card': show_card, 'show_loyal': show_loyal,
                                                   'delta': timedelta_slider, 'global_stops': global_stops})   


display(ui3, ui1, ui2, vis)

Active cars:  {1}


Output()

In [7]:
print(car_raw_df[['CarID','name', 'CurrentEmploymentType']].sort_values(by=['CurrentEmploymentType', 'CarID']))
print(car_raw_df[['CarID','name', 'CurrentEmploymentTitle']].sort_values(by=['CurrentEmploymentTitle', 'CarID']))

    CarID                name   CurrentEmploymentType
1     2.0          Lars Azada             Engineering
2     3.0         Felix Balas             Engineering
6     7.0     Isande Borrasca             Engineering
8     9.0         Axel Calzas             Engineering
10   11.0        Gustav Cazar             Engineering
13   14.0       Lidelse Dedos             Engineering
17   18.0     Birgitta Frente             Engineering
18   19.0         Vira Frente             Engineering
24   25.0       Adra Nubarron             Engineering
25   26.0          Marin Onda             Engineering
26   27.0         Kare Orilla             Engineering
27   28.0         Elsa Orilla             Engineering
32   33.0     Brand Tempestad             Engineering
3     4.0     Ingrid Barranco               Executive
9    10.0  Ada Campo-Corrente               Executive
30   31.0   Sten Sanjorge Jr.               Executive
31   32.0         Orhan Strum               Executive
34   35.0   Willem Vasco-Pai

In [8]:
print(card_raw_df[(card_raw_df['car_id'] == 1) & (card_raw_df['location'] == 'Hippokampos')])

              timestamp     location  price FirstName LastName  hour  day  \
55  2014-01-06 13:21:00  Hippokampos  28.23     Lucas  Alcazar    13    6   
333 2014-01-08 13:43:00  Hippokampos  39.80     Lucas  Alcazar    13    8   
643 2014-01-11 13:37:00  Hippokampos  75.62     Lucas  Alcazar    13   11   
714 2014-01-12 14:06:00  Hippokampos  71.99     Lucas  Alcazar    14   12   
813 2014-01-13 13:28:00  Hippokampos  30.51     Lucas  Alcazar    13   13   

              name car_id  
55   Lucas Alcazar      1  
333  Lucas Alcazar      1  
643  Lucas Alcazar      1  
714  Lucas Alcazar      1  
813  Lucas Alcazar      1  


In [9]:
print(loyal_raw_df[(loyal_raw_df['car_id'] == 1) & (loyal_raw_df['location'] == 'Hippokampos')])

     timestamp     location  price FirstName LastName  day           name  \
474 2014-01-08  Hippokampos  39.80     Lucas  Alcazar    8  Lucas Alcazar   
787 2014-01-12  Hippokampos  31.99     Lucas  Alcazar   12  Lucas Alcazar   
831 2014-01-13  Hippokampos  30.51     Lucas  Alcazar   13  Lucas Alcazar   

    car_id  
474      1  
787      1  
831      1  
